In [1]:
from sys import path
from os.path import dirname as dir

In [2]:
path.append(dir(path[0]))

In [3]:
import yaml
from AgentLayer.RLAgents.A2C import A2C
from AgentLayer.Environment.PortfolioEnv import PortfolioEnv
from FinancialDataLayer.DataCollection.DataDownloader import DataDownloader
from FinancialDataLayer.DataProcessing.DefaultFeatureEngineer import DefaultFeatureEngineer
from AgentLayer.DataSplitter.TimeSeriesSplitter import TimeSeriesSplitter

/Users/barisozakar/opt/anaconda3/envs/uniFi/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# IMPORT .yaml FILE
# Gather user parameters
with open("..//user_params.yaml", "r") as stream:
    try:
        user_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

tickers = user_params["tickers"]
env_kwargs = user_params["env_params"]
train_params = user_params["train_params"]
policy_params = user_params["policy_params"]
test_params = user_params["test_params"]

In [5]:
# FETCH DATA
print("\nTest 3: Downloading from Yahoo.........")
downloaded_df = DataDownloader(start_date='2009-01-01',
                               end_date='2021-10-31',
                               ticker_list=tickers).download_from_yahoo()


Test 3: Downloading from Yahoo.........
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (12924, 8)


In [6]:
downloaded_df.head()

,date,open,high,low,close,volume,tic,day
0,2008-12-31,3.070357,3.133571,3.047857,2.606278,607541200,AAPL,2
1,2008-12-31,41.590000,43.049999,41.500000,32.005882,5443100,BA,2
2,2008-12-31,43.700001,45.099998,43.700001,30.628809,6277400,CAT,2
3,2008-12-31,72.900002,74.629997,72.900002,43.314419,9964300,CVX,2
4,2009-01-02,3.067143,3.251429,3.041429,2.771174,746015200,AAPL,4


In [7]:
# PREPROCESS DATA
print("\nTest 4: Feature engineer.........")

df_processed = DefaultFeatureEngineer(use_default=False,
                                      tech_indicator_list=env_kwargs["tech_indicator_list"],
                                      use_covar=True,
                                      use_return=True,
                                      use_vix=True,
                                      use_turbulence=True).extend_data(
    downloaded_df)  # included technical indicators as features



Test 4: Feature engineer.........
Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3231, 8)
Successfully added vix
Successfully added turbulence index
Successfully added covariances
Successfully added returns


In [8]:
df_processed.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,cov_list,return_list
0,2010-12-31,AAPL,11.533929,11.552857,11.475357,9.849808,193508000.0,4.0,0.091907,9.973600,9.686132,58.973180,56.042158,4.076954,9.738817,9.566210,17.750000,0.439976,"[[0.0002841357152231862, 0.0001814046402739405...",tic AAPL BA CAT ...
0,2010-12-31,BA,64.900002,65.290001,64.620003,52.123985,2137400.0,4.0,-0.163487,53.158962,50.459634,49.173982,30.865687,6.670857,51.751258,53.392654,17.750000,0.439976,"[[0.0002841357152231862, 0.0001814046402739405...",tic AAPL BA CAT ...
0,2010-12-31,CAT,93.830002,93.900002,93.309998,68.765221,2542700.0,4.0,1.742872,70.643151,64.949739,68.265975,72.930039,45.749416,65.923377,62.360929,17.750000,0.439976,"[[0.0002841357152231862, 0.0001814046402739405...",tic AAPL BA CAT ...
0,2010-12-31,CVX,91.580002,91.800003,91.000000,57.546185,5152900.0,4.0,1.203228,58.549676,53.227386,65.105887,107.259779,53.406336,54.660563,53.596117,17.750000,0.439976,"[[0.0002841357152231862, 0.0001814046402739405...",tic AAPL BA CAT ...
1,2011-01-03,AAPL,11.630000,11.795000,11.601429,10.063862,445138400.0,0.0,0.101104,10.012171,9.684601,62.862028,142.930261,25.488753,9.760334,9.586746,17.610001,2.340943,"[[0.0002849614375050196, 0.0001806944895954656...",tic AAPL BA CAT ...


In [11]:
# split data to train and test
splitter = TimeSeriesSplitter()
train = splitter.get_split_data(df_processed, '2009-01-01', '2020-06-30')
trade = splitter.get_split_data(df_processed, '2020-07-01', '2021-09-02')

In [12]:
# CREATE TRAIN ENV
env = PortfolioEnv(df=train, **env_kwargs)
env_train, _ = env.get_env()

In [13]:
# CREATE A2C AGENT
a2c = A2C(env=env_train, **policy_params["A2C_PARAMS"])

In [14]:
# TRAIN A2C AGENT
a2c.train_model(**train_params["A2C_PARAMS"])

In [15]:
# CREATE TEST ENV
env_test = PortfolioEnv(df=trade, **env_kwargs)

In [16]:
# TEST A2C AGENT
df_daily_return_a2c, df_actions_a2c = a2c.predict(environment=env_test, **test_params["A2C_PARAMS"])

            0
0    0.000000
1    0.007676
2    0.015928
3   -0.022620
4    0.007470
..        ...
291 -0.014079
292  0.012880
293 -0.000889
294 -0.005995
295 -0.008121

[296 rows x 1 columns]
begin_total_asset:1000000
end_total_asset:1492026.8855532568
Sharpe:  1.5448194466164042
hit end!


In [17]:
a2c_weights = a2c.extract_weights(df_actions_a2c)

In [18]:
print("Portfolio weights per day:\n")
a2c_weights

Portfolio weights per day:



,date,weights
0,2020-07-01,tic weight 0 AAPL 0.25 1 BA 0.2...
1,2020-07-02,tic weight 0 AAPL 0.158234 1 BA 0...
2,2020-07-06,tic weight 0 AAPL 0.155071 1 BA 0...
3,2020-07-07,tic weight 0 AAPL 0.158113 1 BA 0...
4,2020-07-08,tic weight 0 AAPL 0.158403 1 BA 0...
...,...,...
291,2021-08-26,tic weight 0 AAPL 0.174776 1 BA 0...
292,2021-08-27,tic weight 0 AAPL 0.185917 1 BA 0...
293,2021-08-30,tic weight 0 AAPL 0.169209 1 BA 0...
294,2021-08-31,tic weight 0 AAPL 0.184807 1 BA 0...


In [19]:
# SAVE AGENT
a2c.save_model("../AgentLayer/RLAgents/a2c_model")

In [20]:
# LOAD AGENT
loaded_a2c_model = a2c.load_model("../AgentLayer/RLAgents/a2c_model")